In [ ]:
# --------------------------------------------------------------
# 1. Imports
# --------------------------------------------------------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --------------------------------------------------------------
# 2. Exact Treasury-Note Duration (semi-annual coupons)
# --------------------------------------------------------------
def tnote_duration(y: float, face: float = 100.0, coupon_rate: float = 0.05,
                  maturity: float = 10.0, freq: int = 2) -> float:
    """
    Compute modified duration of a Treasury Note using discrete discounting.
    
    Parameters
    ----------
    y : float
        Annual yield to maturity (decimal, e.g., 0.043 for 4.3%).
    face : float
        Face value (default 100).
    coupon_rate : float
        Annual coupon rate (default 5%, typical for illustration).
    maturity : float
        Years to maturity (default 10).
    freq : int
        Coupon frequency per year (2 = semi-annual).
    
    Returns
    -------
    float
        Modified duration in years.
    """
    if y == 0:
        # Limit case: duration → maturity (flat price)
        return maturity

    period_yield = y / freq
    periods = int(maturity * freq)
    coupon_per_period = (coupon_rate / freq) * face

    # Time points (in years)
    t = np.arange(1, periods + 1) / freq

    # Cash flows
    cf = np.full(periods, coupon_per_period)
    cf[-1] += face  # add principal at maturity

    # Present values
    pv = cf / (1 + period_yield) ** np.arange(1, periods + 1)

    # Price
    price = pv.sum()

    # Weighted average time (Macaulay duration)
    mac_duration = (t * pv).sum() / price

    # Modified duration
    mod_duration = mac_duration / (1 + period_yield)

    return mod_duration


# Vectorize for array input
tnote_duration_vec = np.vectorize(tnote_duration)


# --------------------------------------------------------------
# 3. Approximation Functions
# --------------------------------------------------------------
# --------------------------------------------------------------
# 3. Approximation Functions (SCALAR + ARRAY SAFE)
# --------------------------------------------------------------
def duration_continuous(y, T: float = 10.0):
    """
    Continuous-time duration: (1 - exp(-yT))/y
    Works with scalar or np.ndarray.
    """
    y = np.asarray(y)
    result = np.full_like(y, fill_value=T, dtype=float)
    nonzero = y != 0
    result[nonzero] = (1 - np.exp(-y[nonzero] * T)) / y[nonzero]
    return result


def duration_log_approx(y_nominal, T: float = 10.0):
    """
    Log-yield approximation: r = log(1+y), then [1 - (1+r)^(-T)] / r
    Works with scalar or np.ndarray.
    """
    y_n = np.asarray(y_nominal)
    result = np.full_like(y_n, fill_value=T, dtype=float)
    nonzero = y_n != 0
    r = np.log(1 + y_n[nonzero])
    result[nonzero] = (1 - np.exp(-r * T)) / r
    return result

# --------------------------------------------------------------
# 4. Generate yield grid and compute all durations
# --------------------------------------------------------------
y_grid = np.linspace(0.0, 0.10, 500)  # 0% to 10%

# Exact T-Note duration (5% coupon, semi-annual)
d_exact = tnote_duration_vec(y_grid)

# Approximations
d_cont = duration_continuous(y_grid)
d_log = duration_log_approx(y_grid)

# --------------------------------------------------------------
# 5. Plotting
# --------------------------------------------------------------
plt.figure(figsize=(14, 8))

# --- Panel 1: Duration curves ---
plt.subplot(2, 2, 1)
plt.plot(y_grid * 100, d_exact, label='Exact T-Note (5% coupon, semi-annual)', linewidth=2.5, color='black')
plt.plot(y_grid * 100, d_cont, label='Continuous approx $D = (1-e^{-yT})/y$', linestyle='--', linewidth=2)
plt.plot(y_grid * 100, d_log, label='Log-yield approx $D = [1-(1+\\log(1+y))^{-T}]/\\log(1+y)$', linestyle=':', linewidth=2)

plt.title('10-Year Treasury-Note Duration vs. Yield', fontsize=14)
plt.xlabel('Yield (%)')
plt.ylabel('Modified Duration (years)')
plt.legend()
plt.grid(alpha=0.3)

# --- Panel 2: Absolute error vs Exact ---
plt.subplot(2, 2, 2)
err_cont = np.abs(d_cont - d_exact)
err_log  = np.abs(d_log  - d_exact)

plt.plot(y_grid * 100, err_cont, label='|Continuous − Exact|', linewidth=2)
plt.plot(y_grid * 100, err_log,  label='|Log-yield − Exact|', linewidth=2, linestyle='--')

plt.title('Absolute Error of Approximations', fontsize=14)
plt.xlabel('Yield (%)')
plt.ylabel('Error (years)')
plt.legend()
plt.grid(alpha=0.3)

# --- Panel 3: Relative error (optional) ---
plt.subplot(2, 2, 3)
rel_err_cont = err_cont / d_exact * 100
rel_err_log  = err_log  / d_exact * 100

plt.plot(y_grid * 100, rel_err_cont, label='Continuous error (%)')
plt.plot(y_grid * 100, rel_err_log,  label='Log-yield error (%)', linestyle='--')

plt.title('Relative Error (%)', fontsize=14)
plt.xlabel('Yield (%)')
plt.ylabel('Relative Error (%)')
plt.legend()
plt.grid(alpha=0.3)

# --- Panel 4: Summary table snippet ---
plt.subplot(2, 2, 4)
y_sample = [0.01, 0.03, 0.05, 0.07, 0.10]
data = []
for y in y_sample:
    row = [
        f"{y*100:.1f}%",
        f"{tnote_duration(y):.3f}",
        f"{duration_continuous(y):.3f}",
        f"{duration_log_approx(y):.3f}",
        f"{abs(duration_continuous(y) - tnote_duration(y)):.3f}",
        f"{abs(duration_log_approx(y) - tnote_duration(y)):.3f}",
    ]
    data.append(row)

df_summary = pd.DataFrame(data, columns=[
    'Yield', 'Exact', 'Cont.Appx', 'Log.Appx', 'Cont.Err', 'Log.Err'
])
plt.axis('off')
table = plt.table(cellText=df_summary.values, colLabels=df_summary.columns,
                  cellLoc='center', loc='center', bbox=[0, 0.1, 1, 0.8])
table.auto_set_font_size(False)
table.set_fontsize(9)
table.scale(1.2, 1.6)
plt.title('Numerical Comparison (Sample Yields)', fontsize=12, pad=20)

plt.tight_layout()
plt.show()